# <center>Bing Image Downloader</center>

Import the required modules and functions

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import os
import re
import csv
from datetime import datetime
from urllib.request import urlretrieve

In [ ]:
# To create an empty csv file if it does not exists already, uncomment the below code
header = ['ImageID','Title','Date']

#with open('BingImages.csv','w',newline='', encoding='UTF8') as f:
#    writer = csv.writer(f)
#    writer.writerow(header)


#with open('Failed.csv','w',newline='', encoding='UTF8') as f:
#    writer = csv.writer(f)
#    writer.writerow(['ImageID'])


Download the homepage and find the number of pages in the website

In [ ]:
homepage_url = 'https://windows10spotlight.com/'

homepage_url_data = requests.get(homepage_url).text

homepage_soup = BeautifulSoup(homepage_url_data,"html.parser")

number_of_pages = int(re.findall('\d+.\d+', homepage_soup.find_all("nav", {"class":"navigation pagination"})[0].text)[1])

home_directory = os.getcwd()

bing_df = pd.read_csv('BingImages.csv')

#countries_df = pd.read_csv('countries.csv')

Download each page in the website to collect the image id's in each of it

Clean the image caption

In [ ]:
def clean_caption(caption):
    return caption.replace('/',' ').replace('\\',' ').replace('?','').replace('"','').replace("'",'')


In [ ]:
#write the image_stamp to the csv
def report_success(id, title, img_date):

    data = [id, title, img_date]
    with open('BingImages.csv','a+',newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(data)


In [ ]:

# to maintain a record of the failed downloads  
def report_failed(id):

    os.chdir(home_directory)
    data = [id]
    with open('Failed.csv','a+',newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(data)


In [ ]:

def download_image_pc(link, caption, extension):

    os.chdir(r'.\Bing Images\Desktop Background')
    file_name = caption + '.' + extension
    urlretrieve(link, file_name)

    #link_data = requests.get(link)
    #path=os.path.join(os.getcwd() , file_name)
   
    #with open(path, 'wb') as f:
    #    print(link)
    #    print(file_name)
    #    f.write(link_data.content)

    os.chdir(home_directory)


#from urllib.request import urlretrieve

#url = 'https://windows10spotlight.com/wp-content/uploads/2022/08/fee98416ec0b1c608573ff159ee7a235.jpg'
#urlretrieve(url, 'image.jpg')


In [ ]:

def download_image_mobile(link, caption, extension):

    os.chdir(r'.\Bing Images\Mobile Background')

    file_name = caption + '.' + extension
    urlretrieve(link, file_name)

    #link_data = requests.get(link)
    #path=os.path.join(os.getcwd() , file_name)

    #with open(path, 'wb') as f:
    #    print(link)
    #    f.write(link_data.content)
    
    os.chdir(home_directory)
    


In [ ]:
def redirect_to_image(image_id):

    try:
        image_page_url = 'https://windows10spotlight.com/images/' + str(image_id)

        image_page_url_data = requests.get(image_page_url).text
        
        image_page_soup = BeautifulSoup(image_page_url_data, 'html.parser')

        #extract the date of the image
        dateStr = image_page_soup.find_all("span", {"class":"date"})[0].text.replace('-',' ')
        image_date = datetime.strptime(dateStr, '%d %b %Y').date()

        #extract the title of the image
        image_caption = clean_caption(image_page_soup.find('h1').text)
        #country = image_caption.split(',')[-1] 

        #extract the download links
        image_soup = image_page_soup.find("div", {"class":"entry"}).find_all('a')

        pc_image_link = image_soup[0].get('href')
        pc_image_extension = pc_image_link.split('.')[-1]

        mobile_image_link = image_soup[1].get('href')
        mobile_image_extension = mobile_image_link.split('.')[-1]


        download_image_pc(pc_image_link, image_caption, pc_image_extension)
        download_image_mobile(mobile_image_link, image_caption, mobile_image_extension)

        #save to the dataframe
        bing_df.loc[len(bing_df.index)] = [image_id, image_caption, image_date]

        print(image_id)
        
        report_success(image_id, image_caption, image_date)

    except:
        print('Error saving: {0}'.format(image_id))
        report_failed(image_id)
        


In [ ]:

def redirect_to_page(page_number):

    page_url = 'https://windows10spotlight.com/page/' + str(page_number)

    page_url_data = requests.get(page_url).text

    page_soup = BeautifulSoup(page_url_data, 'html.parser')

    page_content_soup = page_soup.find(id='content')
    
    for link in page_content_soup.find_all('h2'):
        if(link.parent.name == 'article'):
            img_id = link.text

            if(img_id not in bing_df['ImageID'].values):
                redirect_to_image(img_id)


In [ ]:
#browse through every page
for page in range(number_of_pages, 0, -1):
    
    print(page)
    redirect_to_page(page) #go to the specific page number